# COLLECTING DATA
---

Nhiệm vụ của phần này là thực hiện lấy dữ liệu từ thông tin imdbid và title trong file ids_titles.txt.

## 1. IMPORT THƯ VIỆN CẦN THIẾT

In [1]:
import pandas as pd
import time
import os
import requests
from bs4 import BeautifulSoup
import re

## 2. THAY ĐỔI PHIÊN LÀM VIỆC

In [2]:
# chia 3 ra nha mina hong no k cho minh lay :D
# từ từ để ngâm cú chứ nhìn này trông cx hơi ngu :v
member_part = {
    "Trinh": {
        "part": 1,
        "api_key": None # đk api xong áp vô nè
    },

    "Thi": {
        "part": 2,
        "api_key": None # đk api xong áp vô nè
    },

    "MinhUyen": {
        "part": 3,
        "api_key": 'd4c83dec'
    } 
}

In [3]:
# Điền tên mn vào đây theo từ điển nè
info = member_part["MinhUyen"]

part = info["part"]
apiKey = info["api_key"]

## 3. ĐỌC DỮ LIỆU ĐƯỢC THU THẬP TRƯỚC ĐÓ ĐỂ TÌM KIẾM THÊM

In [12]:
file_name = f"../Data/pre_data_part{part}.csv"

In [15]:
pre_data = pd.read_csv(file_name)
pre_data.dtypes

Id/Title        object
Release year     int64
Age rating      object
Runtime         object
Genre           object
dtype: object

## 4. HÀM LẤY DỮ LIỆU CỦA TRANG `OMDB API`

Hàm này trả về link từ id hoặc tiêu đề phim

In [16]:
def convert_url(item: str, year:int,  key = apiKey):
    if item.startswith("tt"):
        url = f'https://www.omdbapi.com/?i={item}&apikey={key}'
    else: 
        title = item.split(":")[-1].strip() # lấy tên sau :
        title = re.sub(r'[^a-zA-Z0-9\s]', '', title) # xóa kí tự đặc biệt
        title = '+'.join(title.split(' ')) # thêm dấu + giữa các từ
        url = f'https://www.omdbapi.com/?t={title}&y={year}&apikey={key}'

    return url

Hàm này thì thu thập dữ liệu

In [28]:
def collectMovieData(pre_data, key = apiKey):
    full_title = []
    full_rated = []
    released_time = []
    full_runtime = []
    full_genre = []
    metaScore = []
    rating = []
    vote = []
    boxOffice = []
    language = []
    award = []

    items = pre_data['Id/Title']
    release_year = pre_data['Release year']
    genre = pre_data['Genre']
    age_rating = pre_data['Age rating']
    runtime = pre_data['Runtime']

    for i in range(len(items)):
        print(f"[LOG]: Collecting data of {items[i]}")
        url = convert_url(items[i], year = release_year[i], key=key)
        trying = 1

        while True:    
            respond = requests.get(url)
            if respond.status_code == 200:
                data = respond.json()
                if data.get("Response") == "False": # bỏ qua những phim không thể tim thấy trong trang
                    break

                full_title.append(data.get('Title', 'N/A'))
                
                full_rated.append(data.get('Rated', age_rating[i]))

                released_time.append(data.get('Released', release_year[i]))

                full_runtime.append(data.get('Runtime', runtime[i]))
                
                full_genre.append(data.get('Genre', genre[i]))
                    
                metaScore.append(data.get('Metascore', 'N/A'))
                rating.append(data.get('imdbRating', 'N/A'))
                vote.append(data.get('imdbVotes', 'N/A'))
                boxOffice.append(data.get('BoxOffice', 'N/A'))
                language.append(data.get('Language', 'N/A'))
                award.append(data.get('Awards', 'N/A'))
                break
                
            else:
                trying += 1
                if trying > 3:
                    print(f"Request fail after 3 retries with code {respond.status_code}")

                time.sleep(2)

    movieDataframe = pd.DataFrame({
            "Title": full_title,
            "Release time": released_time,
            "Age rating": full_rated,
            "Runtime": full_runtime,
            "Genre": full_genre,
            "Lanuage": language,
            "Awards": award,
            "Metascore": metaScore,
            "IMDb Rating": rating,
            "IMDb Votes": vote,
            "Domestic Revenue": boxOffice
        })

    return movieDataframe

này là thu thập dữ liệu nè

In [29]:
df1 = collectMovieData(pre_data, key=apiKey)
df1

[LOG]: Collecting data of tt28097834
[LOG]: Collecting data of tt10651230
[LOG]: Collecting data of The Burial
[LOG]: Collecting data of tt21807222
[LOG]: Collecting data of tt23334616


,Title,Release time,Age rating,Runtime,Genre,Lanuage,Awards,Metascore,IMDb Rating,IMDb Votes,Domestic Revenue
0,The Mission,08 Dec 2023,PG-13,103 min,Documentary,English,11 nominations,N/A,6.4,"1,075","$58,377"
1,The Peasants,13 Oct 2023,R,114 min,"Animation, Drama, History","Polish, Latin",10 wins & 19 nominations,N/A,7.6,"6,678","$155,188"
2,The Burial,13 Oct 2023,R,126 min,Drama,English,2 wins & 4 nominations,N/A,7.1,"37,620",N/A
3,Saw X,29 Sep 2023,R,118 min,"Horror, Mystery, Thriller","English, Spanish",15 nominations,N/A,6.6,"84,301","$53,607,898"
4,Last Call for Istanbul,24 Nov 2023,TV-MA,91 min,"Drama, Romance",Turkish,1 win,N/A,5.3,"9,358",N/A


## 4. LƯU DỮ LIỆU THÔ VÀO FILE CSV

In [30]:
file_name = f"../Data/movie_data_part{str(part)}.csv"
df1.to_csv(file_name)